## import Liberaries

In [2]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np 
import warnings
warnings.filterwarnings('ignore')

In [3]:
data=pd.read_csv('Hotel Reservations.csv')
data.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


In [3]:
data.drop('Booking_ID',axis=1,inplace=True)

In [4]:
#check duplicates
data.duplicated().sum()

In [5]:
data.shape

In [6]:
data.drop_duplicates(inplace=True)

In [7]:
data.index

In [8]:
data.reset_index(inplace=True)
data.head()

In [9]:
data.drop('index',inplace=True,axis=1)

In [10]:
data.head()

In [11]:
def column_info(data):
    cols=[]
    dtypes=[]
    n_uniques=[]
    v_uniques=[]
    null_per=[]
    
    for col in data.columns:
        cols.append(col)
        dtypes.append(data[col].dtype)
        v_uniques.append(data[col].unique())
        n_uniques.append(data[col].nunique())
        null_per.append(data[col].isna().sum()/len(data))
        
    return pd.DataFrame({'column':cols,'dtype':dtypes,'unique values':v_uniques,'n_unique':n_uniques,'null per':null_per})


In [12]:
column_info(data)

In [13]:
data[data['no_of_adults']==0]['booking_status'].value_counts()

In [14]:
data[data['no_of_adults']==0]['no_of_children'].value_counts()

In [15]:
#no_of_childern
data[data['no_of_children']==10]

In [16]:
data[data['no_of_children']==9]

In [17]:
data['no_of_children'].value_counts()

In [18]:
data['no_of_children'].describe()

In [19]:
data['no_of_children'].replace(10,3,inplace=True)
data['no_of_children'].replace(9,3,inplace=True)

In [20]:
column_info(data)

In [21]:
data[data['no_of_weekend_nights']==0]['no_of_week_nights'].value_counts()

In [22]:
for i in range(len(data)):
    if data.loc[i,'no_of_weekend_nights']==0 and data.loc[i,'no_of_week_nights']==0:
        data.loc[i,'no_of_week_nights']=1

In [23]:
data[data['no_of_weekend_nights']==0]['no_of_week_nights'].value_counts()

In [24]:
data['lead_time'].describe()

In [25]:
#avg_price_per_room
plt.figure(figsize=(8,6))
sns.distplot(data['avg_price_per_room'])
plt.show()

In [26]:
column_info(data)

In [27]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data['booking_status']=le.fit_transform(data['booking_status'])

In [28]:
cat_features=[]
for col in data.columns:
    if data[col].dtype==object:
        cat_features.append(col)
cat_features

In [29]:
import pickle

In [30]:
for col in cat_features:
    le=LabelEncoder()
    data[col]=le.fit_transform(data[col])
    
    pickle.dump(le,open("{}_le.pkl".format(col),"wb"))

In [31]:
column_info(data)

In [32]:
data.columns

In [33]:
data.shape

In [34]:
#booking status
plt.figure(figsize=(8,6))
plt.pie(x=data['booking_status'].value_counts(),labels=['Not canceled','Canceled'],explode=[0.0,0.2],autopct='%0.2f%%',shadow=True)
plt.show()

In [35]:
X=data.drop('booking_status',axis=1).values
y=data['booking_status'].values
from sklearn.model_selection import train_test_split
train_X,test_X,train_y,test_y=train_test_split(X,y,test_size=0.2)

In [36]:
from sklearn.preprocessing import StandardScaler
sd=StandardScaler()
train_X=sd.fit_transform(train_X)
test_X=sd.transform(test_X)
pickle.dump(sd,open("scaler.pkl","wb"))

In [37]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(class_weight='balanced')
model.fit(train_X,train_y)

In [38]:
from sklearn.metrics import accuracy_score,recall_score,precision_score,confusion_matrix,f1_score
def evaluate_model(X,y,model):
    y_pred=model.predict(X)
    
    #acc
    acc=accuracy_score(y_pred,y)
    #f1 score
    f1=f1_score(y_pred,y)
    #recall 
    recall=recall_score(y_pred,y)
    #precision 
    precision=precision_score(y_pred,y)
    #
    cm=confusion_matrix(y_pred,y)
    
    tn,fp,fn,tp=cm.ravel()
    specificty=tn/(tn+fp)
    return pd.Series({'Accuracy':acc,"F1 score":f1,'recall':recall,'precesion':precision,'specificty':specificty})

In [39]:
evaluate_model(train_X,train_y,model)

In [40]:
evaluate_model(test_X,test_y,model)

In [41]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(class_weight='balanced',max_depth=10)
model.fit(train_X,train_y)

In [42]:
evaluate_model(train_X,train_y,model)

In [43]:
evaluate_model(test_X,test_y,model)


In [44]:
import pickle 
pickle.dump(model,open('model.pkl','wb'))

In [4]:
data.columns

Index(['Booking_ID', 'no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'type_of_meal_plan', 'required_car_parking_space',
       'room_type_reserved', 'lead_time', 'arrival_year', 'arrival_month',
       'arrival_date', 'market_segment_type', 'repeated_guest',
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
       'avg_price_per_room', 'no_of_special_requests', 'booking_status'],
      dtype='object')